# Extract Categories from FIFA-Texts

In [8]:
path = '/home/bfh/irsed/daten/FIFA/'

In [9]:
from bs4 import BeautifulSoup
import requests

In [10]:
web = 'https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response = requests.get(web)
content = response.text
soup = BeautifulSoup(content, 'lxml')

In [11]:
# Find the subheadings - they can serve as superordinate categories
lines = soup.find('table', class_='infobox')
headers = lines.find_all('th', class_='infobox-header')

for h in headers:
    print(h)

<th class="infobox-header" colspan="2">Tournament details</th>
<th class="infobox-header" colspan="2">Final positions</th>
<th class="infobox-header" colspan="2">Tournament statistics</th>


In [12]:
# Find the subordinate categories
lines = soup.find('table', class_='infobox')
headers = lines.find_all('tr')

for h in headers:
    s = h.getText()
    print(str(s))

Copa do Mundo da FIFA Brasil 2014 (Brazilian Portuguese)
Juntos num só ritmo(lit. 'Together in a single rhythm')
Tournament details
Host countryBrazil
Dates12 June – 13 July
Teams32 (from 5 confederations)
Venue(s)12 (in 12 host cities)
Final positions
Champions Germany (4th title)
Runners-up Argentina
Third place Netherlands
Fourth place Brazil
Tournament statistics
Matches played64
Goals scored171 (2.67 per match)
Attendance3,429,873 (53,592 per match)
Top scorer(s) James Rodríguez(6 goals)[1]
Best player(s) Lionel Messi[2]
Best young player Paul Pogba[3]
Best goalkeeper Manuel Neuer[4]
Fair play award Colombia[5]
← 2010 2018 → 


In [13]:

def clean_text(content):
    try:
        start = content.index('[')  #exption, if missing
        end = content.index(']')
        text = content[0:start]
        count = content[start+1:end]
        try:
            z = int(count)
            content = text
            # skip footnotes e.g. [42]
        except:
            pass
    except:
        pass
    return content

In [14]:
def clean_text(content):
    # skipt footn otes in []
    try:
        start = content.index('[')  #exption, if missing
        end = content.index(']')
        text = content[0:start]
        count = content[start+1:end]
        try:
            z = int(count)
            content = text
            # skip footnotes e.g. [42]
        except:
            pass
    except:
        pass
    # Information after the opening round brackets is no longer relevant
    try:
        start = content.index('(')
        content = content[0:start]
    except:
        pass
    return content.strip()



def one_document(year, document, categories):
    # Get the document again
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    # Analysis of the input shows that the tr tags must be read in sequence.
    lines = soup.find('table', class_='infobox')
    trs = lines.find_all('tr')
    header = ""
    text = ""
    for tr in trs:
        label = tr.find('th', class_='infobox-label')
        content = tr.find('td', class_='infobox-data')
        newheader = tr.find('th', class_='infobox-header')
        if newheader != None:
            header = newheader.getText()
        if label != None:
            label = label.getText()
            content = tr.find('td', class_='infobox-data')
            text = ""
            if content != None:
                content = content.getText()
            v = clean_text(content)
            category = header + '/' + label
            categories.add(category)
            text += category+"="+content
            create_field(document, category, content)
            text=""


def create_field (document, fieldname, text):
    if len(text.strip()) > 0:
        field = xml.new_tag('field', attrs = {'name':fieldname})
        field.string = text
        document.append(field)


"""
extract single categories
"""
categories = set()


# Define the years we are interested in
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018, 2022]
# years = [1958, 2014]

xml = BeautifulSoup(features='xml')
add = xml.new_tag('add')
xml.append(add)

# For each year we create a new doc-tab
for year in years:
    print(year)
    document = xml.new_tag('doc')
    create_field(document, "year", str(year))   # id generate field id
    one_document(year,document, categories)
    xml.add.append(document)

categories = list(categories)
categories.sort()
for cat in categories:
    print(cat)
with open(path+"fifa_categories.csv", "w") as f:
    for category in categories:
        f.write(category+"\n")

# Und jetzt noch schön herausschreiben
from bs4.formatter import XMLFormatter
formatter = XMLFormatter(indent=0)
with open(path+"fifa_categories.xml", "w") as f:
    output = str(xml)
    output.replace('\n',' ')
    print(output)
    f.write(output)

1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
2022
Final positions/Champions
Final positions/Fourth place
Final positions/Runners-up
Final positions/Third place
Tournament details/Dates
Tournament details/Host countries
Tournament details/Host country
Tournament details/Teams
Tournament details/Venue(s)
Tournament statistics/Attendance
Tournament statistics/Best goalkeeper
Tournament statistics/Best player(s)
Tournament statistics/Best young player
Tournament statistics/Fair play award
Tournament statistics/Goals scored
Tournament statistics/Matches played
Tournament statistics/Top scorer(s)
<?xml version="1.0" encoding="utf-8"?>
<add><doc><field name="year">1930</field><field name="Tournament details/Host country">Uruguay</field><field name="Tournament details/Dates">13–30 July</field><field name="Tournament details/Teams">13 (from 3 confederations)</field><field name="Tournament details/Venue(s)">3 (in 1 host city)</field><fi